In [5]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import cross_validation
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot    
import plotly.graph_objs as go
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop
from sklearn.cross_validation import train_test_split
import keras
from random import randint

import random

# --- GENSIM PACKAGE ---
import gensim, logging
from gensim.models import Word2Vec, doc2vec, Doc2Vec

from gensim.models import doc2vec
from collections import namedtuple


%matplotlib inline


### function to get doc2vec

In [6]:
def doc_to_vec(doc):
    doc1 = []
    doc1.append(doc)
    docs = []
    analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
    for i, text in enumerate(doc1):
        words = text.lower().split()
        tags = [i]
        docs.append(analyzedDocument(words, tags))

    # Train model (set min_count = 1, if you want the model to work with the provided example data set)

    model = doc2vec.Doc2Vec(docs, size = 200, window = 300, min_count = 1, workers = 4)

    # Get the vectors

    s = model.docvecs
    return s

### loading data

In [8]:
train_v1 = pd.read_json('./data/train-v1.1.json')
test_v1 = pd.read_json('./data/dev-v1.1.json')
squad_train = pd.read_json('./data/squad_train_doc.json')
squad_test = pd.read_json('./data/squad_dev_doc.json')

In [9]:
data_train = squad_train
data_train.rename(columns={'passages': 'documents'}, inplace=True)


In [10]:
data_train.head()

,documents,title
0,"[{'context': 'Architecturally, the school has ...",University_of_Notre_Dame
1,[{'context': 'Beyoncé Giselle Knowles-Carter (...,Beyoncé
2,[{'context': 'Montana i/mɒnˈtænə/ is a state i...,Montana
3,"[{'context': 'The phrase ""in whole or in part""...",Genocide
4,[{'context': 'The emergence of resistance of b...,Antibiotics


### Document list

In [11]:
title_list = [data_train.title[i] for i in range(data_train.shape[0])]

### Segregating Dataframe

In [12]:
## Returns list of all context and list of all question sets

def get_compact_dataframe():
    context_list = []
    question_list = []
    
    for doc in data_train.documents:
        context_list.append(get_each_context_each_questionSet(doc)[0])
        question_list.append(get_each_context_each_questionSet(doc)[1])
    
    return context_list, question_list

In [13]:
## Pass a document and it returns list of context and list of set of questions

def get_each_context_each_questionSet(document):
    each_doc_context_list = [document[i]['context'] for i in range(len(document))]
    each_doc_question_list = [document[i]['questions'] for i in range(len(document))]
    return  each_doc_context_list, each_doc_question_list

In [14]:
context_list, question_list = get_compact_dataframe()[0], get_compact_dataframe()[1]
compact_dataframe = pd.DataFrame({'title':title_list, 'context':context_list, 'questions': question_list})

## DATAFRAME

In [15]:
compact_dataframe.head()

,context,questions,title
0,"[Architecturally, the school has a Catholic ch...",[[What sits on top of the Main Building at Not...,University_of_Notre_Dame
1,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,"[[What was Beyoncé's role in Destiny's Child?,...",Beyoncé
2,[Montana i/mɒnˈtænə/ is a state in the Western...,"[[Where does the state's name come from?, What...",Montana
3,"[The phrase ""in whole or in part"" has been sub...",[[Which phrase is especially contentious withi...,Genocide
4,[The emergence of resistance of bacteria to an...,"[[What does emergence of resistance reflect?, ...",Antibiotics


In [16]:
compact_dataframe[compact_dataframe.title == '2008_Sichuan_earthquake']

,context,questions,title
10,[The 2008 Sichuan earthquake or the Great Sich...,[[In what year did the earthquake in Sichuan o...,2008_Sichuan_earthquake


### Total questions

In [17]:
list_questions_perDOC = []
count = 0

for each_doc_question in compact_dataframe.questions:

    Doc_questions = []
    for each_Set in each_doc_question:
        for each_question in each_Set:
            Doc_questions.append(each_question)
            count +=1
    
    list_questions_perDOC.append(Doc_questions)

### Implementing Gensim 

In [18]:
class DocIterator(object):
    
    ## Initailizes document's list(doc1,doc2...) and its label's list('Book_1','Book_2',...)
    def __init__(self, doc_list, labels_list):
        self.doc_list = doc_list
        self.labels_list = labels_list
    
    ## Assigns label1 to a list of all words in doc1, label2 to all words in doc2, etc.
    def __iter__(self):
        
        for idx, doc in enumerate(self.doc_list):
            yield doc2vec.LabeledSentence(words=doc.split(), tags=[self.labels_list[idx]])

In [19]:
## Contains the whole data

data = []
docLabels = title_list

for i in docLabels:
    x = compact_dataframe[compact_dataframe.title == i].context.sum()
    y = ' '.join(x)
    data.append(y)

In [20]:
iter_docs = DocIterator(data, docLabels)

### Model

In [21]:
model = Doc2Vec(iter_docs, size=200, window=50, min_count=1, workers=11, alpha=0.025, min_alpha=0.025, dbow_words = 1, iter = 20)

### creating dummy dataframe using five questions from 5 documents

In [22]:
questions_list = ['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?','In what city did Beyonce grow up?',
                 'How many ranges are part of the Rocky Mountains?',
                  'What 2001 case was declared genocide by the International Criminal Tribune for the Former Yugoslavia (ICTY)?',
                 'What does emergence of resistance reflect?']

In [23]:
documents_doc2vec = []
for i in range(5):
    documents_doc2vec.append(model.docvecs[i])


In [24]:
data_d = []
for i in range(5):
    for j in range(5):
        data_d.append(title_list[j])  

In [25]:
data_q = []
for i in range(5):
    for j in range(5):
        data_q.append(questions_list[i])

In [26]:
def scores():
    scores = []
    for i in range(25):
        scores.append(random.random())
    return scores    

In [27]:
s1 = scores()
s2 = scores()
s3 = scores()
s4 = scores()

In [28]:
print(randint(2,25))

21


In [29]:
random.sample(range(5), 1)

[1]

In [30]:
y =[]
rank = []
for i in range(5):
    k = 2
    for j in range(5):
        if(i==j):
            y.append(1)
            rank.append(1)
        else:
            y.append(0)
            rank.append(k)
            k += 1
rank

[1, 2, 3, 4, 5, 2, 1, 3, 4, 5, 2, 3, 1, 4, 5, 2, 3, 4, 1, 5, 2, 3, 4, 5, 1]

In [31]:
q_vs_d = pd.DataFrame(
    {'questions': data_q,
     'documents': data_d,
     's1': s1,
     's2': s2,
     's3': s3,
     's4': s4,
     'y': y,
     'rank':rank
    })

In [32]:
q_vs_d

,documents,questions,rank,s1,s2,s3,s4,y
0,University_of_Notre_Dame,To whom did the Virgin Mary allegedly appear i...,1,0.292092,0.097499,0.192597,0.736611,1
1,Beyoncé,To whom did the Virgin Mary allegedly appear i...,2,0.788528,0.678500,0.768750,0.947541,0
2,Montana,To whom did the Virgin Mary allegedly appear i...,3,0.273200,0.439731,0.083541,0.228149,0
3,Genocide,To whom did the Virgin Mary allegedly appear i...,4,0.740361,0.111123,0.720138,0.327966,0
4,Antibiotics,To whom did the Virgin Mary allegedly appear i...,5,0.679090,0.627296,0.268764,0.190431,0
5,University_of_Notre_Dame,In what city did Beyonce grow up?,2,0.692027,0.595484,0.565558,0.727373,0
6,Beyoncé,In what city did Beyonce grow up?,1,0.243540,0.216696,0.319605,0.638034,1
7,Montana,In what city did Beyonce grow up?,3,0.722053,0.639608,0.550363,0.829222,0
8,Genocide,In what city did Beyonce grow up?,4,0.835313,0.312631,0.553184,0.373930,0
9,Antibiotics,In what city did Beyonce grow up?,5,0.385470,0.335298,0.578177,0.316151,0


In [33]:
def inputq(question):
    df1 = q_vs_d[((q_vs_d['questions'] == question)& (q_vs_d['rank'] == 1))]
    df2 = q_vs_d[((q_vs_d['questions'] == question)& (q_vs_d['rank'] == max(rank)))]  
    df3 = q_vs_d[((q_vs_d['questions'] == question)& (q_vs_d['rank'] == randint(2,4)))] 
    new_df = df1.append(df2, ignore_index=True)
    new_df2 =new_df.append(df3, ignore_index=True)
    for i in range(3):
        new_df2['questions'][i] = doc_to_vec(question)[0]
        new_df2['documents'][i] = model.docvecs[title_list.index(new_df2['documents'][i])]
    y_train = new_df2['y']
    x_train2 = new_df2[['s1','s2','s3','s4']] 
    new_df2.drop(['y','rank','s1','s2','s3','s4'],axis = 1, inplace = True)
    x_train = new_df2
    return x_train,y_train,x_train2
    

In [35]:
x_train,y_train,x_train2 = inputq('To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?')

/home/user/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/user/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [92]:
xtrain =xtrain.reshape()

(2,)

In [120]:
train_x_array = []
for each in x_train.as_matrix():
    for nested in each:
        train_x_array.append(nested)

train_x_array = np.array(train_x_array)

In [124]:
train_x_array = train_x_array.reshape(3,2,200)

In [126]:
train_x_array.shape

(3, 2, 200)

In [136]:
y_train = np.array(y_train)

In [137]:
y_train

array([1, 0, 0])

In [127]:
model = Sequential()

In [128]:
model.add(Dense(500, input_shape=(2,200)))
model.add(Dense(600, activation='relu'))
model.add(Flatten())
model.add(Dense(1))

In [139]:
model.compile(optimizer=RMSprop(),loss='binary_crossentropy',metrics = ['accuracy'])

In [140]:
model.fit(train_x_array, y_train, batch_size=1, epochs=5, validation_split = .2, verbose=2)

Train on 2 samples, validate on 1 samples
Epoch 1/5
0s - loss: 8.0590 - acc: 0.0000e+00 - val_loss: 1.0000e-07 - val_acc: 0.0000e+00
Epoch 2/5
0s - loss: 8.0590 - acc: 0.0000e+00 - val_loss: 1.0000e-07 - val_acc: 0.0000e+00
Epoch 3/5
0s - loss: 8.0590 - acc: 0.0000e+00 - val_loss: 1.0000e-07 - val_acc: 0.0000e+00
Epoch 4/5
0s - loss: 8.0590 - acc: 0.0000e+00 - val_loss: 1.0000e-07 - val_acc: 0.0000e+00
Epoch 5/5
0s - loss: 8.0590 - acc: 0.0000e+00 - val_loss: 1.0000e-07 - val_acc: 0.0000e+00


In [134]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 2, 500)            100500    
_________________________________________________________________
dense_25 (Dense)             (None, 2, 600)            300600    
_________________________________________________________________
flatten_8 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 1201      
Total params: 402,301
Trainable params: 402,301
Non-trainable params: 0
_________________________________________________________________


In [132]:
x_train2

,s1,s2,s3,s4
0,0.292092,0.097499,0.192597,0.736611
1,0.679090,0.627296,0.268764,0.190431
2,0.273200,0.439731,0.083541,0.228149


In [38]:
ytrain = y_train.values.tolist()
ytrain = keras.utils.to_categorical(ytrain, 2)

z= []


In [40]:
x_train.shape

(3, 2)

In [45]:
xtrain = np.array(x_train)
xtrain2 = np.array(x_train2)
xtrain2

array([[ 0.29209162,  0.09749882,  0.19259699,  0.7366112 ],
       [ 0.6790903 ,  0.6272955 ,  0.26876447,  0.19043088],
       [ 0.27320041,  0.43973055,  0.08354065,  0.22814886]])

In [43]:
xtrain

array([[ array([ 2.01394415,  0.03585275,  1.3813417 ,  7.96615076,  7.35949659,
        0.91579366,  2.17811036, -6.35143614, -2.52540374, -3.50507188,
        0.55032665, -1.11314905,  1.92313516,  0.56830662, -0.28139034,
        1.84442759, -7.77372408, -6.94920301,  4.10894537,  0.99260843,
       -2.8056128 ,  6.80880165, -0.59066927,  0.56344479, -6.87310028,
       -1.51675236,  2.84560275, -4.27219439,  1.5247618 ,  3.08890057,
       -3.30584073,  1.06405914, -2.98729897, -3.33614278,  3.06652236,
        0.58145165, -6.27639866,  0.74633253, -3.8253634 , -1.54233944,
       -3.94853854,  2.96002793, -0.240844  , -4.73850727,  1.42409801,
       -4.9074235 ,  5.15057898,  0.54581958,  1.21229005,  3.86014795,
        5.2014122 , -1.81772888, -1.57212996, -1.61680365, -0.17548494,
        3.0380187 , -2.1900959 ,  3.40655947,  2.22727418,  1.08796191,
       -0.5415169 , -1.1212703 ,  0.31352213,  0.07798663, -6.45791388,
       -3.90841269, -7.1050539 ,  5.8700695 ,  0.596674

In [44]:
xtrain.reshape(3,2,1)

array([[[ array([ 2.01394415,  0.03585275,  1.3813417 ,  7.96615076,  7.35949659,
        0.91579366,  2.17811036, -6.35143614, -2.52540374, -3.50507188,
        0.55032665, -1.11314905,  1.92313516,  0.56830662, -0.28139034,
        1.84442759, -7.77372408, -6.94920301,  4.10894537,  0.99260843,
       -2.8056128 ,  6.80880165, -0.59066927,  0.56344479, -6.87310028,
       -1.51675236,  2.84560275, -4.27219439,  1.5247618 ,  3.08890057,
       -3.30584073,  1.06405914, -2.98729897, -3.33614278,  3.06652236,
        0.58145165, -6.27639866,  0.74633253, -3.8253634 , -1.54233944,
       -3.94853854,  2.96002793, -0.240844  , -4.73850727,  1.42409801,
       -4.9074235 ,  5.15057898,  0.54581958,  1.21229005,  3.86014795,
        5.2014122 , -1.81772888, -1.57212996, -1.61680365, -0.17548494,
        3.0380187 , -2.1900959 ,  3.40655947,  2.22727418,  1.08796191,
       -0.5415169 , -1.1212703 ,  0.31352213,  0.07798663, -6.45791388,
       -3.90841269, -7.1050539 ,  5.8700695 ,  0.59667

In [472]:
xtrain.shape

(3, 2)

In [436]:
l = []
for i in range(3):
    l1 = []
    for j in range(2):
        for k in xtrain[i][j]:
            l1.append(k)
    l.append(l1)
            

In [441]:
qd_array = np.array(l)

In [443]:
qd_array.shape[1]

400

200

In [483]:
model_scores = Sequential()
model_scores.add(Dense(600, activation='relu', input_shape=(200,3)))
model_scores.add(Dense(500, activation='relu'))

model_scores.add(Dense(ytrain.shape[1], activation='softmax'))
model_scores.compile(optimizer=RMSprop(),loss='categorical_crossentropy',metrics = ['accuracy'])
model_scores = model_scores.fit(xtrain, ytrain, batch_size=1, epochs=5, validation_split = .2, verbose=2)

ValueError: Error when checking model input: expected dense_118_input to have 3 dimensions, but got array with shape (3, 2)